In [1]:
import humana_preprocessing as hf
import sagemaker
import os

import boto3

In [2]:
import humana_package.constants as params

In [3]:
bucket = "humana-data"
train_input = 's3://{}/rawdata/original_raw_files/train'.format(bucket)
train_input

's3://humana-data/rawdata/original_raw_files/train'

In [4]:
%run humana_preprocessing.py --train_or_test="train" --debugger=True \
--input_data_path='s3://humana-data/rawdata/original_raw_files/train'

['s3://humana-data/rawdata/original_raw_files/train/condition.csv', 's3://humana-data/rawdata/original_raw_files/train/credit.csv', 's3://humana-data/rawdata/original_raw_files/train/dependent.csv']
The files used for processing the train data ['s3://humana-data/rawdata/original_raw_files/train/condition.csv', 's3://humana-data/rawdata/original_raw_files/train/credit.csv', 's3://humana-data/rawdata/original_raw_files/train/dependent.csv']
Successfully imported data from S3. Shape of the train data (10000, 438)
0    0
1    0
2    0
3    0
4    0
Name: transportation_issues, dtype: int64
Successfully imported data from S3. Shape of the train data (10000, 438)
Preprocessed train data
Saved the files


In [6]:
import humana_package.constants as params

In [10]:
import pandas as pd
df = pd.read_csv("data/df_fe_train.csv", header=None)

feature_names = params.fe_column_names
label_column = params.label_column
if len(df.columns) == len(feature_names) + 1:
    # This is a labelled example, includes the ring label
    df.columns = feature_names + [label_column]
elif len(df.columns) == len(feature_names):
    # This is an unlabelled example.
    df.columns = feature_names
df.head()

,credit_bal_1stmtgcredit_60dpd,credit_bal_agencyfirstmtg_60dpd,credit_bal_heloc_60dpd,credit_bal_nonagnfirstmtg_60dpd,credit_bal_nonmtgcredit_60dpd,credit_bal_studentloan_60dpd,credit_bal_totalallcredit_60dpd,credit_bal_autobank,credit_bal_autofinance,credit_bal_consumerfinance,...,sor_ind_sum,sor_pmpm_sum,trm_ind_sum,trm_pmpm_sum,vco_ind_sum,vco_pmpm_sum,total_ind,total_pmpm,service_bool,transportation_issues
0,864.263515,364.200415,9.100900,569.012631,3059.660249,3215.762022,5817.811692,2245.510843,2172.577397,699.143043,...,0.0,0.000000,0.0,0.0,4.0,1.286296,22,6.954966,1,0
1,429.012668,438.835063,21.462771,129.242818,1633.195247,633.297258,2248.812860,4950.330338,2470.968265,836.815438,...,1.0,0.083333,0.0,0.0,3.0,0.333333,11,1.083333,1,0
2,487.244759,215.293670,9.060431,449.407714,1469.197512,1165.968943,2357.211774,4227.609499,5634.157054,1496.651571,...,1.0,1.000000,0.0,0.0,1.0,0.583333,19,5.833333,1,0
3,701.522802,206.697739,35.318230,445.190344,1698.740262,613.688796,2801.548062,4964.892445,8015.374205,474.326537,...,1.0,0.616381,0.0,0.0,0.0,0.000000,21,7.751455,1,0
4,1822.958518,1887.606611,2.255284,1337.204541,3361.830981,1445.861795,5315.380440,5890.979925,8189.115348,1008.685155,...,0.0,0.000000,0.0,0.0,3.0,1.000000,34,19.736661,1,0


In [13]:
df[params.label_column] = df[params.label_column].astype(params.label_columns_dtype[params.label_column])

In [6]:
import joblib

In [7]:
sc = joblib.load("data/std_scaler.joblib")

FileNotFoundError: [Errno 2] No such file or directory: 'data/std_scaler.joblib'

In [ ]:
numeric_cols = [col for col, type_ in params.feature_columns_dtype.items() if type_ != "category"]
category_cols = [col for col, type_ in params.feature_columns_dtype.items() if type_ == "category"]

assert len(sc.transformers_[0][2]) == len(numeric_cols)
assert len(sc.transformers_[1][2]) == len(category_cols)

col_names = list(sc.transformers_[0][1].named_steps['scaler'].get_feature_names_out(numeric_cols)) + list( sc.transformers_[1][1].named_steps['onehot'].get_feature_names_out(category_cols))

In [ ]:
len(col_names)

In [ ]:
# %run humana_transportation_featurizer.py --train_or_test="test" --debugger=True \
# --input_data_path='s3://humana-data/rawdata/original_raw_files/valid'

In [ ]:
# bucket = "humana-data"
# valid_input = 's3://{}/rawdata/original_raw_files/valid'.format(bucket)
# valid_input

In [ ]:
# !pip install xgboost

In [ ]:
def feature_importance_fn(xgb, col_names):
    feature_importance = pd.DataFrame({'feature':col_names, 'importance':xgb.feature_importances_}).sort_values('importance',ascending=False).reset_index().drop(columns='index')
    feature_importance.sort_values(by='importance',ascending=False)
    fig, ax = plt.subplots()
    fig.set_size_inches(8.27,15)
    plt.title('Feature Importance Plot')
    sns.barplot(x='importance',y='feature',ax=ax,data=feature_importance[:60])
    return feature_importance


In [ ]:
bst = load("data/model.joblib")

In [ ]:
import shap

import matplotlib.pyplot as plt
import seaborn as sns

import humana_package.constants as params

In [ ]:
assert (len(bst.get_booster().feature_names) == len(col_names))

In [ ]:
top_features_xg = feature_importance_fn(bst, col_names)